In [21]:
import requests
import time
import os
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

In [22]:
# !pip install duckduckgo-images-api
from duckduckgo_images_api import search

In [ ]:
# Test if images make sense
results = search("search")

img_urls = [r['image'] for r in results['results']]

f, axes = plt.subplots(2, 2, figsize=(15, 15))
for i in range(2):
    for j in range(2):
        img_data = requests.get(img_urls[:4][i+2*j])
        img_data.raise_for_status()
        img = Image.open(BytesIO(img_data.content))        
        axes[i][j].imshow(img)
        axes[i][j].axis("off")
plt.show()

In [23]:
def export_imgs(search_terms, n_train_imgs, size):
    
    imgs_added = set() # keep this to minimize duplicates
    train_i = 0 # keep count to control images that get added to train and test sets
    test_i = 0
    
    for st in range(len(search_terms)):

        # Get image URLs for search_term (default is 100 images)
        results = search(search_terms[st])
        img_urls = [r['image'] for r in results['results']]


        # Clean up image urls and add to respective folder 
        for url in img_urls:
            
            # After n_train_imgs images have been added to training set, add the remaining images to the test set
            if train_i >= n_train_imgs*(st+1):
                is_train = False
            else:
                is_train = True
                
            if is_train:
                # Create path to store images
                dir_path = f"./train/"
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)
            else:
                dir_path = f"./test/"
                if not os.path.exists(dir_path):
                    os.makedirs(dir_path)

            ## this will check if there is a '?' or '!' after the '.jpg' and get rid of them
            split = url.split("/")

            last_item = split[-1]

            second_split = last_item.split("?")

            if len(second_split) > 1:
                last_item = second_split[0]

            third_split = last_item.split("!")

            if len(third_split) > 1:
                last_item = third_split[0]      
            ##

            # Save images with resize
            if is_train:
                path = os.path.join(dir_path, f'{train_i}.jpg')
            else:
                path = os.path.join(dir_path, f'{test_i}.jpg')
            if last_item not in imgs_added:
                try:
                    img_data = requests.get(url)
                    #image_data.raise_for_status()
                    img = Image.open(BytesIO(img_data.content))
                    img_resized = img.resize(size)
                    img_resized.save(path)
                    if is_train:
                        train_i += 1 
                    else:
                        test_i += 1 # keep track of images added
                except OSError:
                    pass
            imgs_added.add(last_item)
            
            
            # Save images without resize
#             path = os.path.join(dir_path, f'{i}.jpg')
#             if last_item not in imgs_added:
#                 try:
#                     with open(path, "wb") as f:
#                         img_data = requests.get(url)
#                         #image_data.raise_for_status()

#                         f.write(img_data.content)
#                         i += 1 # keep track of images added
#                 except OSError:
#                     pass
#             imgs_added.add(last_item)

In [24]:
qbs_terms = ['tom brady', 'patrick mahomes']
n_train = 60
resize_size = (800, 600)

export_imgs(qbs_terms, n_train, resize_size)

DEBUG:duckduckgo_images_api.api:Hitting DuckDuckGo for Token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): duckduckgo.com:443
DEBUG:urllib3.connectionpool:https://duckduckgo.com:443 "POST / HTTP/1.1" 200 None
DEBUG:duckduckgo_images_api.api:Obtained Token
DEBUG:duckduckgo_images_api.api:Hitting Url : https://duckduckgo.com/i.js
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): duckduckgo.com:443
DEBUG:urllib3.connectionpool:https://duckduckgo.com:443 "GET /i.js?l=us-en&o=json&q=tom+brady&vqd=3-122622667198778323765620601299424308411-325966634408261230851748717290239902219&f=%2C%2C%2C&p=1&v7exp=a HTTP/1.1" 200 None
DEBUG:duckduckgo_images_api.api:Hitting Url Success : https://duckduckgo.com/i.js
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): duckduckgo.com:443


Width 3779, Height 2834
Thumbnail https://tse2.mm.bing.net/th?id=OIP.JE37IRJYuMdjM4WsHXeVlQHaFj&pid=Api
Url https://insurancenewsnet.com/innarticle/7-tips-to-pull-prospects-in-inspired-by-tom-brady
Title b'7 Tips To Pull Prospects In, Inspired By Tom Brady - InsuranceNewsNet'
Image https://insurancenewsnet.com/wp-content/uploads/2018/06/636646098321192280-tom-brady.jpg
__________
Width 2000, Height 1333
Thumbnail https://tse3.mm.bing.net/th?id=OIP.8n_1czwWKRohQLqZAiuZfQHaE7&pid=Api
Url https://nypost.com/2020/11/24/tom-brady-had-an-eventful-inefficient-night-in-loss-to-rams/
Title b"Tom Brady had an eventful, 'inefficient' night in loss to Rams"
Image https://i2.wp.com/nypost.com/wp-content/uploads/sites/2/2020/11/tom-brady-02.jpg?quality=90&strip=all&ssl=1
__________
Width 1200, Height 800
Thumbnail https://tse2.mm.bing.net/th?id=OIP.afzsETAOkIH_Rw1-0oElwQHaE8&pid=Api
Url https://dknation.draftkings.com/2020/3/18/21184935/tom-brady-signing-with-buccaneers-super-bowl-odds-win-total-nfc

DEBUG:urllib3.connectionpool:https://duckduckgo.com:443 "GET /i.js?q=tom%20brady&o=json&p=1&s=100&u=bing&f=,,,&l=us-en&l=us-en&o=json&q=tom+brady&vqd=3-122622667198778323765620601299424308411-325966634408261230851748717290239902219&f=%2C%2C%2C&p=1&v7exp=a HTTP/1.1" 200 None
DEBUG:duckduckgo_images_api.api:Hitting Url Success : https://duckduckgo.com/i.js?q=tom%20brady&o=json&p=1&s=100&u=bing&f=,,,&l=us-en
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): duckduckgo.com:443


Width 4716, Height 3077
Thumbnail https://tse3.mm.bing.net/th?id=OIP.UHCjokER9Qpx5m7fVeriZwHaE1&pid=Api
Url https://heavy.com/sports/2015/01/watch-tom-brady-press-conference-live-stream-online-deflategate/
Title b'WATCH: Tom Brady Press Conference Live Stream Online | Heavy.com'
Image https://heavy.com/wp-content/uploads/2015/01/460874570.jpg?quality=65&strip=all
__________
Width 2000, Height 1334
Thumbnail https://tse4.mm.bing.net/th?id=OIP.AJXAnkw5FH9KpX_WUREcFAHaE8&pid=Api
Url https://musketfire.com/2019/08/31/tom-brady-changing-helmets-big-deal/
Title b'Tom Brady details why changing helmets is a big deal'
Image https://images2.minutemediacdn.com/image/fetch/w_2000,h_2000,c_fit/https://musketfire.com/wp-content/uploads/getty-images/2016/04/899136004.jpeg
__________
Width 1890, Height 1063
Thumbnail https://tse1.mm.bing.net/th?id=OIP.zAdT3UHxy9pY7tFxinfSwgHaEK&pid=Api
Url https://heightwiki.org/wheres-tom-brady-today-bio-wife-net-worth-kids-salary-family-career/
Title b"Where's Tom 

DEBUG:urllib3.connectionpool:https://duckduckgo.com:443 "GET /i.js?q=tom%20brady&o=json&p=1&s=200&u=bing&f=,,,&l=us-en&l=us-en&o=json&q=tom+brady&vqd=3-122622667198778323765620601299424308411-325966634408261230851748717290239902219&f=%2C%2C%2C&p=1&v7exp=a HTTP/1.1" 200 None
DEBUG:duckduckgo_images_api.api:Hitting Url Success : https://duckduckgo.com/i.js?q=tom%20brady&o=json&p=1&s=200&u=bing&f=,,,&l=us-en
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): duckduckgo.com:443


...

Width 705, Height 881
Thumbnail https://tse1.mm.bing.net/th?id=OIP.YrSdMSyh-cA6EIB9mV4p1gHaJQ&pid=Api
Url https://news.yahoo.com/photos-patrick-mahomes-brittany-matthews-200154568.html
Title b"Photos of Patrick Mahomes and Brittany Matthews' Daughter, Sterling Skye"
Image https://s.yimg.com/ny/api/res/1.2/JR.VHiBBIf_bU4sxxUxKVw--/YXBwaWQ9aGlnaGxhbmRlcjt3PTcwNTtoPTg4MS4yNQ--/https://s.yimg.com/uu/api/res/1.2/t4cMwjo6tT1zWN4GvWOCIw--~B/aD0xMzUwO3c9MTA4MDthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/people_218/b4da8803cdb485dc282fba8cffb52985
__________
Width 696, Height 694
Thumbnail https://tse1.mm.bing.net/th?id=OIP.HNNXCWt9ZD0EwFDAN9bHDwHaHY&pid=Api
Url https://www.webofbio.com/mia-randall-mahomes/
Title b"Who is Patrick Mahomes II's Sister Mia Randall Mahomes?"
Image https://www.webofbio.com/wp-content/uploads/2021/03/Mia-Randall-Mahomes-Family-696x694.jpg
__________
Width 1080, Height 1080
Thumbnail https://tse1.mm.bing.net/th?id=OIP.9Xo6_ftJMMp9_kvMMa8ZDwHaHa&pid=Api
Url https:

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.webofbio.com:443
DEBUG:urllib3.connectionpool:https://www.webofbio.com:443 "GET /wp-content/uploads/2021/03/Mia-Randall-Mahomes-Family-696x694.jpg HTTP/1.1" 200 63236
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): i.pinimg.com:443
DEBUG:urllib3.connectionpool:https://i.pinimg.com:443 "GET /originals/6a/6e/82/6a6e825ef6853aca95cc255ccc1cfc94.jpg HTTP/1.1" 200 216564
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): akns-images.eonline.com:443
DEBUG:urllib3.connectionpool:https://akns-images.eonline.com:443 "GET /eol_images/Entire_Site/202082/rs_634x1024-200902060304-634-Patrick-Mahomes-Brittany-Matthews-LT-090220-GettyImages-1126566554.jpg?fit=around%7C634:1024&output-quality=90&crop=634:1024;center,top HTTP/1.1" 200 157960
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.brides.com:443
DEBUG:urllib3.connectionpool:https://www.brides.com:443 "GET /thmb/lEVtZHhYDv827aBuZD

DEBUG:urllib3.connectionpool:https://blacksportsonline.com:443 "GET /wp-content/uploads/2020/07/Patrick-Mahomes-Brittany-Matthews-1.jpg HTTP/1.1" 403 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.lifeandstylemag.com:443
DEBUG:urllib3.connectionpool:https://www.lifeandstylemag.com:443 "GET /wp-content/uploads/2020/11/brittanylynne_59964327_609251152892776_5363295667732296446_n.jpg?w=820&resize=820%2C1024 HTTP/1.1" 200 118218
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.birthdaywiki.com:443
DEBUG:urllib3.connectionpool:https://www.birthdaywiki.com:443 "GET /uploads/25216/brittany-matthews-instagrambrittanylynne7.jpg HTTP/1.1" 301 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): birthdaywiki.com:443
DEBUG:urllib3.connectionpool:https://birthdaywiki.com:443 "GET /uploads/25216/brittany-matthews-instagrambrittanylynne7.jpg HTTP/1.1" 200 40931
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn.justjared.co

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): blackchristiannews.com:443
DEBUG:urllib3.connectionpool:https://blackchristiannews.com:443 "GET /wp-content/uploads/2020/10/122094883_997505074082467_6413421160917182200_n-696x696.jpg HTTP/1.1" 200 93039
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): media.okmagazine.com:443
DEBUG:urllib3.connectionpool:https://media.okmagazine.com:443 "GET /brand-img/qYtHkaZyp/0x0/patrick-mahomes-mom-randi-tweets-instagram-slams-gisele-bundchen-refs-super-bowl-kansas-city-chiefs-1612777690753.jpg HTTP/1.1" 200 103799
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s.yimg.com:443
DEBUG:urllib3.connectionpool:https://s.yimg.com:443 "GET /ny/api/res/1.2/AWsg5iLpwpw4T7IZSDA5oA--/YXBwaWQ9aGlnaGxhbmRlcjt3PTcwNTtoPTcxNi45NDkxNTI1NDIzNzI5/https://s.yimg.com/uu/api/res/1.2/z8k0yy_5qF2JG7Wm.Xs9dg--~B/aD0xNTAwO3c9MTQ3NTthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/people_218/1a39c4a2ba473e2819e4067637dfa9e3 HTTP/

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): imagesvc.meredithcorp.io:443
DEBUG:urllib3.connectionpool:https://imagesvc.meredithcorp.io:443 "GET /v3/mm/image?url=https:%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F20%2F2021%2F07%2F05%2Fsterling-skye-2.jpg HTTP/1.1" 301 201
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.people.com:443
DEBUG:urllib3.connectionpool:https://www.people.com:443 "GET /thmb/BEN390dqrHjKn078lUvMgIEm-bM=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/sterling-skye-2-d48a9bd1821243d5800afac4f3e9a8c7.jpg HTTP/1.1" 200 155323
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn.fansided.com:443
DEBUG:urllib3.connectionpool:https://cdn.fansided.com:443 "GET /wp-content/uploads/usat-images/2017/05/10034620-nfl-kansas-city-chiefs-patrick-mahomes-press-conference-850x560.jpeg HTTP/1.1" 200 122316
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.dailyhawker.com:443
DEBUG:urlli

In [ ]:
# Using makesense.ai labelling tool:
    # clean images - delete duplicates, label all faces and label 'other' if not a qb + draw bounding boxes
# create dataset with all labels (qb + bounding box info)